I want to join all the prior and post together!

In [9]:
import pickle
pickle_in = open("Best 10 features in Actor", "rb")
actor_indices = pickle.load(pickle_in)
pickle_in.close()
pickle_in1 = open("Best 10 features in Partner", "rb")
partner_indices = pickle.load(pickle_in1)



In [21]:
import pandas as pd
import numpy as np

missing_values = ["n/a", "na", "--", " "]
actor_train = pd.read_csv("/home/cardosoo/492_19summer/MachineLearning/Data/Training actor.csv", na_values = missing_values )
actor_test = pd.read_csv("/home/cardosoo/492_19summer/MachineLearning/Data/Testing actor.csv" , na_values = missing_values )
partner_train = pd.read_csv("/home/cardosoo/492_19summer/MachineLearning/Data/Testing partner.csv" , na_values = missing_values )
partner_test = pd.read_csv("/home/cardosoo/492_19summer/MachineLearning/Data/Testing partner.csv" , na_values = missing_values )



In [11]:
"""
Given train and test this function finds intersecting columns names
returns a sorted lowercase list of the intersecting column names
"""
def intersect_columsn(train_df,test_df):
    train = train_df.fillna(0)
    test = test_df.fillna(0)
    train.isna().sum().sum(),test.isna().sum().sum()

    # columns may have same names but different cases
    train_cols = [x.lower() for x in train.columns.tolist()] 
    test_cols = [x.lower() for x in test.columns.tolist()]
    print(len(train_cols))
    print(len(test_cols))
    A = set(train_cols)
    B = set(test_cols)
    C = A & B
    C = A & B

    len(C)

    print("Non intersecting column names: ")
    print("A-C",A - C)
    print("B-C",B-C)
    D = A - C
    E = B - C
    Not_C = D | E
    for x in Not_C:
        if x in train_cols:
            train_cols.remove(x)
        if x in test_cols:
            test_cols.remove(x)
    print(len(train_cols),len(test_cols))

    train_X = train_cols[0:len(train_cols)-1]# not including digging actor
    y = train_cols[len(train_cols)-1]
    test_X = test_cols[0:len(test_cols)-1]
    train_X.sort()
    test_X.sort()
    train_X.append(y)
    test_X.append(y) 
    print(train_X == test_X)

    train_cols = train.columns.tolist() 
    test_cols = test.columns.tolist()
    # making the column names lowercase so we can index the features we want
    train_cols = [x.lower() for x in train_cols]
    test_cols = [x.lower() for x in test_cols]

    #reassigning column names
     
    return (train_cols, test_cols),(train_X, test_X)
     


In [26]:
(actor_train.columns, actor_test.columns), (train_X, test_X) = intersect_columsn(actor_train,actor_test)
actor_train = actor_train[train_X]
actor_test = actor_test[test_X]
(partner_train.columns, partner_test.columns), (train_X, test_X) = intersect_columsn(partner_train,partner_test)
partner_train = partner_train[train_X]
partner_test = partner_test[test_X]

85
85
Non intersecting column names: 
A-C set()
B-C set()
85 85
True
87
87
Non intersecting column names: 
A-C set()
B-C set()
87 87
True


Now i have to find the column names associated with the top 10 features for actor and partner

In [27]:
CR = list(actor_indices[0])
actor_CR_feature = [actor_test.columns.tolist()[i]for i in CR]
CR = list(partner_indices[0])
partner_CR_feature = [partner_test.columns.tolist()[i]for i in CR]

In [14]:
partner_CR_feature

['agreeable',
 'anxious',
 'growth',
 'indepselconstrual',
 'init2',
 'init3',
 'ipdom',
 'ipearnambit',
 'ipintell',
 'ipphyshot']

In [15]:
actor_CR_feature

['asintell',
 'fourminpliked',
 'fourminwarm',
 'init1',
 'ludus',
 'rejectionsen',
 'satpeop',
 'sexor',
 'storge',
 'youint_1']

In [22]:
post_actor_train = pd.read_csv("/home/cardosoo/492_19summer/MachineLearning/Data/Level 2 Post-Interaction predicting Actor, Sample B.csv", na_values = missing_values )
post_actor_test = pd.read_csv("/home/cardosoo/492_19summer/MachineLearning/Data/Level 2 Post-Interaction predicting Actor, Sample B.csv" , na_values = missing_values )


In [16]:
pickle_in = open("Best 10 Post features in Actor", "rb")
post_actor_indices = pickle.load(pickle_in)

In [28]:
(post_actor_train.columns, post_actor_test.columns), (train_X, test_X) = intersect_columsn(post_actor_train,post_actor_test)
post_actor_train = post_actor_train[train_X]
post_actor_test = post_actor_test[test_X]


42
42
Non intersecting column names: 
A-C set()
B-C set()
42 42
True


In [30]:
CR = list(post_actor_indices[0])
post_actor_CR_feature = [post_actor_test.columns.tolist()[i]for i in CR]
post_actor_CR_feature

['chemisactorgm',
 'dominactorgm',
 'intellactorgm',
 'pdiggingactorgm',
 'percpopactorgm',
 'physhotactorgm',
 'psanxactorgm',
 'psavoidactorgm',
 'vitalityactorgm',
 'warmthactorgm']

In [8]:
len(train_X), len(test_X )
train = train[train_X]
test = test[test_X]
train_data = train.values.astype('float')
test_data = test.values.astype('float')
cols = train_data.shape[1]
y = train_data[:,cols-1]
X = train_data[:,0:cols-1]
print(y.shape,X.shape, train_data.shape)

# Joining sample A and sample B together
full_data = np.concatenate((train_data,test_data))
full_data.shape,train_data.shape,test_data.shape

(1092,) (1092, 36) (1092, 37)


((2050, 37), (1092, 37), (958, 37))

# Helper functions

In [11]:
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras import layers
from keras import models
"""
Builds a nn regression modelall_val_mae_histories, all_val_loss_histories, all_loss_histories, all_mae_histories
"""
def build_model(units,layerz,metric,shape):
    model = models.Sequential()
    model.add(layers.Dense(units, activation = 'relu', 
                           input_shape = (shape,)))
    for layer in range(layerz):
        model.add(layers.Dense(units, activation = 'relu'))
    model.add(layers.Dense(1)) #linear layer
    model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = metric  )
    return model

"""
params: units, layerz, metric,num_epochs
returns : mae, loss, validation loss and validition mae histories for each fold
          tuple( all_val_mae_histories, all_val_loss_histories, all_loss_histories, all_mae_histories)
"""
def validaton(hyper_parameters,shape,data):
    k = 4
    
    units, layerz = hyper_parameters['units'], hyper_parameters['layerz']
    metric,num_epochs = hyper_parameters['metric'], hyper_parameters['num_epochs']
    train_data,train_targets = data['train'],data['target']
    
    num_val_samples = len(train_data) // k
    all_val_mae_histories = []
    all_val_loss_histories = []
    all_loss_histories = []
    all_mae_histories = []
    all_val_mape_histories = []
    
    print("val_samples: " + str(num_val_samples))
    for i in range(k):
        print('\tprocessing fold #', i)
        start = i*num_val_samples
        stop = (i + 1) * num_val_samples
        val_data = train_data[start:stop]
        val_target = train_targets[start:stop]

        partial_train_data = np.concatenate( (train_data[:start], train_data[stop:]) ,axis = 0)
        partial_train_target = np.concatenate( (train_targets[:start],train_targets[stop:]), axis = 0) 

        model = build_model(units, layerz, metric,shape)
        history = model.fit(partial_train_data, partial_train_target, epochs = num_epochs, batch_size = 1, verbose = 0,
                            validation_data = (val_data, val_target) )


        val_mae_history = history.history['val_mean_absolute_error']
        val_loss_history = history.history['val_loss']
        loss_history = history.history['loss']
        mae_history = history.history['mean_absolute_error']
        val_mape_history = history.history['val_mean_absolute_percentage_error']
        
        all_val_mae_histories.append(val_mae_history)  
        all_val_loss_histories.append(val_loss_history)
        all_loss_histories.append(loss_history)
        all_mae_histories.append(mae_history)
        all_val_mape_histories.append(val_mape_history)
        
    return all_val_mae_histories, all_val_loss_histories, all_loss_histories, all_mae_histories,all_val_mape_histories

"""
returns the histories of all the P * K folds
"""
def iterated_validation(num_iterations,hyper_parameters,shape,data):
    all_val_mae_histories = []
    all_val_loss_histories = []
    all_loss_histories = []
    all_mae_histories = []
    all_val_mape_histories = []
    train_data,train_targets = data['train'],data['target']
    for i in range(num_iterations):
        print("iteration: "+str(i+1))
            # shuffle training data
        rows = np.arange(train_targets.size)
        indexes = shuffle(rows)
        train_data = train_data[indexes]
        train_targets = train_targets[indexes]
        print("\tStarting indexes for training " +str(indexes[0:5]))
        
        histories = validaton(hyper_parameters,shape,data)
        all_val_mae_histories.extend(histories[0])
        all_val_loss_histories.extend(histories[1])
        all_loss_histories.extend(histories[2])
        all_mae_histories.extend(histories[3])
        all_mape_histories.extend(histories[4])
        
    return all_val_mae_histories, all_val_loss_histories, all_loss_histories, all_mae_histories,all_val_mape_histories

"""
Returns the mean for each elements returned in by the 
iterated_validation or validation function
"""
def average_folds(all_val_mae_histories, all_val_loss_histories, all_loss_histories, all_mae_histories,all_val_mape_histories):
    averages = []
    num_epochs = len(all_val_mae_histories[0])
    averages.append([np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)])
    averages.append([np.mean([x[i] for x in all_val_mae_histories]) for i in range(num_epochs)])
    averages.append([np.mean([x[i] for x in all_loss_histories]) for i in range(num_epochs)])
    averages.append([np.mean([x[i] for x in all_val_loss_histories]) for i in range(num_epochs)])
    averages.append([np.mean([x[i] for x in all_val_mape_histories]) for i in range(num_epochs)])
    return averages

"""
Creates and plots 4 subplots. That represents 
the loss and Mae for the training and validation set
"""
def plots(histories):
    val_mae = 1
    val_loss = 3
    val_mape = 4
    indices = [val_mae,val_mape]
    fig, axes = plt.subplot(1,2,figsize=(12,12))
    axes.reshape((2,))
    fig.title('overfitting with different size features')
    fig.set_xlabel('Epochs')
    fig.set_ylabel('Validation mae')
    color = ["b","r"]
    name = ["MAE","MAPE"]
    for i , axis in enumerate(axes):
        axis.plot(histories[indices[i]], colour[i], label)
        plt.legend()
    print(name + " had a " + str(np.min(histories[val_mae])) +" MAE value")
    print(name + " had a " + str(np.max(histories[val_mape])) +" MAPE value")
    print(name + " had a " + str(np.min(histories[val_loss])) +" Loss value")
    print()
    
    
def plot_features(features, names, smooth=0):
    val_mae_index = 1
    val_loss_index = 3
    epochs = range(1,len(features[names[0]][val_mae_index])+1)
    color = ["b","r","g","k"]
    plt.figure(figsize=(12,12))
    plt.title('overfitting with different size features')
    plt.xlabel('Epochs')
    plt.ylabel('Validation mae')
    for i,name in enumerate(names):
        if smooth != 0:
            plt.plot(epochs, smooth_curve(features[names[i]][val_mae_index]), color[i], label=name) 
        else:
            plt.plot(epochs, features[names[i]][val_mae_index], color[i], label=name) 
        plt.legend()
        print(name + " had a " + str(np.min(features[names[i]][val_mae_index])) +" MAE value")
        print(name + " had a " + str(np.min(features[names[i]][val_loss_index])) +" Loss value")
        print()
    plt.show()
    
"""
Manipulates the data such that it looks smoother when you plot it
"""
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point*(1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points


def normalize(X, indices):
    ratio = 0.8
    split = int(np.floor(X.shape[0]*ratio))
    normalized_train_datas = []
    normalized_test_datas = []
    mean = X[0:split,:].mean(axis = 0)
    normalized_X = X.copy()
    normalized_X[0:split,:] -= mean
    std = X.std(axis = 0)
    normalized_X /= std


    for i in range(len(indices)): # should change this the test_data has information about the whole dataset
        normalized_train_datas.append(normalized_X[0:split,indices[i]])
        normalized_test_datas.append(normalized_X[split:,indices[i]])
    return normalized_train_datas

# Experiment 1:
units = 48, layerz = 3



In [ ]:
num = 10 #num of selected features
#corrolation, Mutual Information, Mean decrease impurities ,Mean decrease accuracy
train_datas = []
test_datas = []
ratio = 0.8
split = int(np.floor(X.shape[0]*ratio))

hyper_parameters = dict(units = 48, layerz = 2, metric =['mae','mape'], num_epochs = 100)
shape = X.shape[1]
train_data = X[0:split,:]
train_target = y[0:split]
data = dict(train = train_data, target = train_target)

histories = iterated_validation(1, hyper_parameters,shape,data)

iteration: 1
	Starting indexes for training [637 382 464 184 243]
val_samples: 218
	processing fold # 0


In [ ]:
#note your loss and accuracy graphs may look different because of initial randomizaiton
pickle_out = open("PostExperiment1RM", "wb")
pickle.dump(features,pickle_out)
pickle_out.close()

In [ ]:
# pickle_in = open("PostExperiment1RM", "rb")
# features = pickle.load(pickle_in)
# pickle_in.close()

In [ ]:
average = average_folds(all_val_mae_histories, all_val_loss_histories, all_loss_histories, all_mae_histories)

In [ ]:
plots(average)